In [1]:
import pandas as pd

In [5]:
matches = pd.read_csv("../../data/Match_Info.csv");

In [6]:
matches.columns

Index(['match_number', 'team1', 'team2', 'match_date', 'toss_winner',
       'toss_decision', 'result', 'eliminator', 'winner', 'player_of_match',
       'venue', 'city', 'team1_players', 'team2_players'],
      dtype='object')

In [7]:
matches.drop(columns=['match_number', 'eliminator', 'team1_players', 'team2_players'], inplace=True)

In [8]:
matches['match_date'] = pd.to_datetime(matches['match_date']).dt.date

We have to amke connection with the teams table in the database, to get team_id

In [9]:
import psycopg2

In [10]:
conn = psycopg2.connect(
    dbname="ipl_dashboard",
    user="ipl_user",
    password="ipl_user_12246",  
    host="localhost",
    port="5432"
)

In [11]:
teams = pd.read_sql("SELECT team_id, name FROM Teams;", conn);

C:\Users\hp\AppData\Local\Temp\ipykernel_9860\1042233605.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  teams = pd.read_sql("SELECT team_id, name FROM Teams;", conn);


In [12]:
print(teams.head())

   team_id                        name
0        1              Mumbai Indians
1        2         Chennai Super Kings
2        3              Delhi Capitals
3        4  Kolkata Knight Respn_iders
4        5                Punjab Kings


In [13]:
team_map = dict(zip(teams['name'], teams['team_id']))

In [ ]:
matches['team1_id'] = matches['team1'].map(team_map)
matches['team2_id'] = matches['team2'].map(team_map)
matches['toss_winner_id'] = matches['toss_winner'].map(team_map)
matches['winner_id'] = matches['winner'].map(team_map)

In [15]:
matches['outcome'] = matches['result'].str.lower()
matches['outcome'] = matches['outcome'].replace({'no result': 'no result', 'tie': 'tie', 'win': 'win'})

In [16]:
players = pd.read_sql("SELECT * FROM Players;", conn)

C:\Users\hp\AppData\Local\Temp\ipykernel_9860\1347944146.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  players = pd.read_sql("SELECT * FROM Players;", conn)


In [17]:
print(players.head())

   player_id              name     fielding_name      batting_name  \
0          1   Ruturaj Gaikwad           Gaikwad        RD Gaikwad   
1          2         Moeen Ali               Ali            MM Ali   
2          3  Aravelly Avanish  Aravelly Avanish  Aravelly Avanish   
3          4     Deepak Chahar            Chahar         DL Chahar   
4          5  Tushar Deshpande         Deshpande      TU Deshpande   

          dob                                            img_url  \
0  1997-01-31  https://img1.hscicdn.com/image/upload/f_auto,t...   
1  1987-06-18  https://img1.hscicdn.com/image/upload/f_auto,t...   
2  2005-06-02  https://img1.hscicdn.com/image/upload/f_auto,t...   
3  1992-08-07  https://img1.hscicdn.com/image/upload/f_auto,t...   
4  1995-05-15  https://img1.hscicdn.com/image/upload/f_auto,t...   

    batting_style       bowling_style      display_name  
0  right-hand bat  right-arm offbreak        RD Gaikwad  
1   left-hand bat  right-arm offbreak            MM Al

In [18]:
player_map = dict(zip(players['display_name'], players['player_id']))  # players must be loaded from DB
matches['player_of_match'] = matches['player_of_match'].map(player_map)

In [19]:
matches['win_margin'] = None
matches['win_type'] = None

In [20]:
matches.rename(columns={
    'match_date': 'date',
    'venue': 'venue',
    'city': 'city'
}, inplace=True)

In [ ]:
final_columns = [
    'date', 'venue', 'city',
    'team1_id', 'team2_id', 'toss_winner_id',
    'toss_decision', 'outcome', 'winner_id',
    'win_margin', 'win_type', 'player_of_match'
]

In [24]:
id_cols = ['team1_id', 'team2_id', 'toss_winner_id', 'winner_id', 'player_of_match']
for col in id_cols:
    matches[col] = matches[col].astype('Int64')

In [27]:
matches['city'] = matches['city'].fillna('Unknown')

In [28]:
matches_final = matches[final_columns]

In [29]:
matches_final.to_csv("cleaned_matches.csv", index=False)